Есть датфрейм с authors, text, annotation и title. Текст предобработан, стоп-слова удалены. У одной статьи может быть несколько авторов, тогда они записаны в одной ячейке authors и разделены запятой. 

Функции:
- для вычисления всех соавторов одного человека по ФИО, они могут быть записаны в разных ячейках датафрейма. 

- функция для векторизации текста.  

- функция для вычисления коэффициента схожести.

- функция для рекомендации новых соавторов. 

В функцию для рекомендации новых соавторов подаётся на вход ФИО автора. На выходе ФИО автора, названия всех статей в которых он участвовал, список его соавторов, список трёх рекомендуемых новых авторов/коллективов авторов, названия их статей и коэффициент близости. 

Ограничения:

- Нельзя рекомендовать автора самому себе 
- Нельзя рекомендовать автору его текущих соавторов. 

Особенности:

Необходимо предусмотреть ситуацию когда рекомендуется несколько человек, написавших одну и ту же статью, при совпадении коэффициента близости у новых соавторов необходимо объединять их ФИО через запятую, писать их общую статью один раз и на 2 и 3 месте писать новых рекомендуемых авторов. При совпадении 2 и 3 рекомендации также следует их объединять и на третье место выводить следующих по коэффициенту схожести коллектив авторов/автора.

In [35]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import numpy as np
import pprint
from gensim.models import Word2Vec

In [37]:
df = pd.read_excel('date_it_lemm.xlsx')

In [39]:
#df['authors'] = df['authors'].str.split(',') #авторов через запятую разделяем

In [41]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Функция для обучения модели Word2Vec на текстах из датафрейма
def train_word2vec_model(texts):
    # Разбиваем тексты на слова
    sentences = [text.split() for text in texts]
    
    # Обучаем модель Word2Vec
    model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
    return model

# Функция для векторизации текста с использованием Word2Vec
def vectorize_text_with_word2vec(text, model):
    words = text.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)  # Усредняем векторы слов
    else:
        return np.zeros(model.vector_size)  # Возвращаем нулевой вектор, если слова не найдены

# Функция для векторизации всех текстов в датафрейме
def vectorize_texts_with_word2vec(df, text_column='text'):
    # Обучаем модель Word2Vec на текстах из датафрейма
    print("Обучение модели Word2Vec...")
    word2vec_model = train_word2vec_model(df[text_column])
    
    # Векторизация текстов
    print("Векторизация текстов...")
    vectors = df[text_column].apply(lambda x: vectorize_text_with_word2vec(x, word2vec_model))
    
    # Преобразуем список векторов в массив
    return np.array(vectors.tolist()), word2vec_model





In [45]:
def recommend_new_coauthors(df, author_name):
    target_mask = df['authors'].str.contains(author_name, regex=False)  # Данные автора
    target_indices = df[target_mask].index.tolist()
    
    if not target_indices:
        return {"error": "Автор не найден"}
    
    current_coauthors = get_coauthors(df, author_name)  # Текущие соавторы
    
    # Векторизация текстов с помощью Word2Vec
    tfidf_matrix, _ = vectorize_texts_with_word2vec(df, text_column='text')
    
    all_indices = df.index.tolist()
    similarities = calculate_similarity(tfidf_matrix, target_indices, all_indices)  # Схожести

    results = pd.DataFrame({
        'index': all_indices,
        'similarity': similarities,
        'authors': df['authors'],
        'title': df['title']
    })
    
    results = results[
        (results['similarity'] < 1.0) &  # Исключаем статьи автора
        (~results['authors'].isin([author_name]))  # Исключаем соавторов
    ]
    
    # Группировка по коэффициенту схожести
    grouped = results.groupby('similarity').agg({
        'authors': lambda x: ', '.join(sorted(set(', '.join(x).split(', ')))),
        'title': lambda x: ', '.join(sorted(set(x)))
    }).reset_index().sort_values('similarity', ascending=False)
    
    recommendations = []
    used_authors = set(current_coauthors + [author_name])  # Формируем рекомендации
    
    for _, row in grouped.iterrows():
        candidates = [a.strip() for a in row['authors'].split(',') if a.strip() not in used_authors]
        if candidates:
            unique_candidates = []
            for candidate in candidates:
                if candidate not in used_authors:
                    unique_candidates.append(candidate)
                    used_authors.add(candidate)
            
            if unique_candidates:
                recommendations.append({
                    'authors': ', '.join(unique_candidates),
                    'titles': row['title'],
                    'similarity': round(row['similarity'], 2)
                })
        
        if len(recommendations) >= 3:
            break
    
    # Итоговый вывод
    output = {
        "Автор": author_name,
        "Текущие соавторы": current_coauthors,
        "Статьи": df[target_mask]['title'].tolist(),
        "Рекомендуемые соавторы": recommendations[:3]
    }
#    save_results_to_txt(author_name, current_coauthors, output["Статьи"], output["Рекомендуемые соавторы"], df)
    
    return output

In [47]:
import pprint

# Пример вызова функции
rec_list = recommend_new_coauthors(df, 'Большаков А. О.')
pprint.pprint(rec_list)

Обучение модели Word2Vec...
Векторизация текстов...
{'Автор': 'Большаков А. О.',
 'Рекомендуемые соавторы': [{'authors': 'Здитовец А. Л.',
                             'similarity': 0.99,
                             'titles': 'ОСНОВНЫЕ ТЕХНОЛОГИИ И ФРЕЙМВОРКИ ДЛЯ '
                                       'БЕКЕНД-РАЗРАБОТКИ НА JAVA'},
                            {'authors': 'Жидков Виталий Алексеевич',
                             'similarity': 0.98,
                             'titles': 'Сравнительный анализ SCADA-систем, '
                                       'применяемых в диспетчерских службах '
                                       'Белгородской энергосистемы'},
                            {'authors': 'Массель Людмила Васильевна, Черноусов '
                                        'Антон Владимирович',
                             'similarity': 0.98,
                             'titles': 'Интеграция унаследованных программных '
                                       'комплексо